In [26]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
f = open("/content/drive/My Drive/Malayalam_NLP/Data/pos_tagged_data/Malayalam_Tagged_Data.txt","r")
data = f.read()
print(len(data))
data = data.split(".        \RD_PUNC")
all_sent = []
for x in data:
    x = x.split("\n")
    v = []
    for y in x:
        if y != "":
            y = y.replace("\\","")
            y = y.split("        ")
            v.append( (y[0],y[1]) )
    v.append(('.','RD_PUNC'))
    all_sent.append(v)
del data
sentences = all_sent
del all_sent
print(len(sentences))

sentences = sentences[:5000]
print(len(sentences))


7041405
29077
5000


In [8]:
import random
sentences = random.sample(sentences, len(sentences)) 
print(len(sentences))

5000


In [0]:
train_test_cutoff = int(.80 * len(sentences)) 
training_sentences = sentences[:train_test_cutoff]
testing_sentences = sentences[train_test_cutoff:]
train_val_cutoff = int(.25 * len(training_sentences))
validation_sentences = training_sentences[:train_val_cutoff]
training_sentences = training_sentences[train_val_cutoff:]

In [0]:
def add_basic_features(sentence_terms, index):
    """ Compute some very basic word features.        :param sentence_terms: [w1, w2, ...] 
        :type sentence_terms: list
        :param index: the index of the word 
        :type index: int
        :return: dict containing features
        :rtype: dict
    """
    term = sentence_terms[index]
    return {
        'nb_terms': len(sentence_terms),
        'term': term,
        'is_first': index == 0,
        'is_last': index == len(sentence_terms) - 1,
        'prefix-1': term[0],
        'prefix-2': term[:2],
        'prefix-3': term[:3],
        'suffix-1': term[-1],
        'suffix-2': term[-2:],
        'suffix-3': term[-3:],
        'prev_word': '' if index == 0 else sentence_terms[index - 1],
        'next_word': '' if index == len(sentence_terms) - 1 else sentence_terms[index + 1],
        'prev_word2': '' if index == 0 else sentence_terms[index - 2],
        'next_word2': '' if index > len(sentence_terms)-3 else sentence_terms[index + 2]

         
    }

In [11]:
print(add_basic_features(sentences[0],1))

{'nb_terms': 3, 'term': ('ശരിയാണ്', 'V_VAUX'), 'is_first': False, 'is_last': False, 'prefix-1': 'ശരിയാണ്', 'prefix-2': ('ശരിയാണ്', 'V_VAUX'), 'prefix-3': ('ശരിയാണ്', 'V_VAUX'), 'suffix-1': 'V_VAUX', 'suffix-2': ('ശരിയാണ്', 'V_VAUX'), 'suffix-3': ('ശരിയാണ്', 'V_VAUX'), 'prev_word': ('ചക്ര', 'N_NN'), 'next_word': ('.', 'RD_PUNC'), 'prev_word2': ('.', 'RD_PUNC'), 'next_word2': ''}


In [0]:
def untag(tagged_sentence):
    """ 
    Remove the tag for each tagged term.:param tagged_sentence: a POS tagged sentence
    :type tagged_sentence: list
    :return: a list of tags
    :rtype: list of strings
    """
    return [w for w, _ in tagged_sentence]
def transform_to_dataset(tagged_sentences):
    """
    Split tagged sentences to X and y datasets and append some basic features.:param tagged_sentences: a list of POS tagged sentences
    :param tagged_sentences: list of list of tuples (term_i, tag_i)
    :return: 
    """
    X, y = [], []
    for pos_tags in tagged_sentences:
        for index, (term, class_) in enumerate(pos_tags):
            # Add basic NLP features for each sentence term
            X.append(add_basic_features(untag(
              pos_tags), index))
            y.append(class_)
    return X, y

In [0]:
X_train, y_train = transform_to_dataset(training_sentences)
X_test, y_test = transform_to_dataset(testing_sentences)
X_val, y_val = transform_to_dataset(validation_sentences)

In [0]:
del training_sentences
del testing_sentences
del validation_sentences
del sentences

In [15]:
print(X_train[1])
print(y_train[1])
print(y_test[1])

{'nb_terms': 14, 'term': 'ജനലുകളിലും', 'is_first': False, 'is_last': False, 'prefix-1': 'ജ', 'prefix-2': 'ജന', 'prefix-3': 'ജനല', 'suffix-1': 'ം', 'suffix-2': 'ും', 'suffix-3': 'ലും', 'prev_word': 'അതിനാല്\u200d', 'next_word': 'വാതിലു', 'prev_word2': '.', 'next_word2': 'കളിലും'}
N_NN
V_VM_VNF


In [0]:
from sklearn.feature_extraction import DictVectorizer# Fit our DictVectorizer with our set of features
dict_vectorizer = DictVectorizer(sparse=False)
dict_vectorizer.fit(X_train + X_test + X_val)# Convert dict features to vectors
X_train = dict_vectorizer.transform(X_train)
X_test = dict_vectorizer.transform(X_test)
X_val = dict_vectorizer.transform(X_val)

In [0]:
from sklearn.preprocessing import LabelEncoder# Fit LabelEncoder with our list of classes
label_encoder = LabelEncoder()
label_encoder.fit(y_train + y_test + y_val)# Encode class values as integers
y3 = y_train
y_train = label_encoder.transform(y_train)
y_test = label_encoder.transform(y_test)
y_val = label_encoder.transform(y_val)

In [0]:

# Convert integers to dummy variables (one hot encoded)

from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
y_val = np_utils.to_categorical(y_val)

In [0]:
print(y_test[1])

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
def build_model(input_dim, hidden_neurons, output_dim):
    """
    Construct, compile and return a Keras model which will be used to fit/predict
    """
    model = Sequential([
        Dense(hidden_neurons, input_dim=input_dim),
        Activation('relu'),
        Dropout(0.2),

        Dense(hidden_neurons),
        Activation('relu'),
        Dropout(0.2),

        Dense(hidden_neurons),
        Activation('relu'),
        Dropout(0.2),
        Dense(output_dim, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

    

In [0]:
print(X_train.shape[1],y_train.shape[1])
print(X_test.shape[1],y_test.shape[1])

59124 33
59124 33


In [0]:
model = build_model(X_train.shape[1],512,y_train.shape[1])
model = model.fit(X_train, y_train, epochs=5, verbose=1)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/5
12913/12913 [==============================] - 109s 8ms/step - loss: 1.1936 - acc: 0.6620
Epoch 2/5
12913/12913 [==============================] - 106s 8ms/step - loss: 0.3349 - acc: 0.9051
Epoch 3/5
12913/12913 [==============================] - 106s 8ms/step - loss: 0.1012 - acc: 0.9725
Epoch 4/5
12913/12913 [==============================] - 105s 8ms/step - loss: 0.0522 - acc: 0.9858
Epoch 5/5
12913/12913 [==============================] - 105s 8ms/step - loss: 0.0337 - acc: 0.9916


In [0]:
from keras.models import load_model

model.model.save('model_v2_2.h5')  # creates a HDF5 file 'my_model.h5'
del model  # deletes the existing model

# returns a compiled model

# identical to the previous one
model = load_model('model_v2_2.h5')

In [0]:
loss, accuracy = model.model.evaluate(X_test, y_test, verbose=1)

 160/4116 [>.............................] - ETA: 4s

/usr/local/lib/python3.6/dist-packages/keras/engine/sequential.py:111: UserWarning: `Sequential.model` is deprecated. `Sequential` is a subclass of `Model`, you can just use your `Sequential` instance directly.
  warnings.warn('`Sequential.model` is deprecated. '


4116/4116 [==============================] - 4s 1ms/step


In [0]:
print(loss, accuracy)

0.8881109667357249 0.8170553936439306
